In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

from sklearn.metrics.pairwise import cosine_similarity

# 문제에 대한 정보
problem = pd.read_csv('data/tag_key.csv')

In [99]:
user = pd.read_csv('data/userSolved.csv')

In [100]:
user

,userId,solved
0,koosaga,1000
1,koosaga,1001
2,koosaga,1002
3,koosaga,1003
4,koosaga,1004
...,...,...
8224055,ou0911,10998
8224056,ou0911,11654
8224057,ou0911,16394
8224058,ou0911,16430


In [101]:
problem.shape

(9722, 198)

In [102]:
problem

,Unnamed: 0.1,Unnamed: 0,problemId,level,averageTries,tag-key,math,implementation,dp,graphs,...,top_tree,chordal_graph,majority_vote,a_star,rb_tree,birthday,degree_sequence,discrete_kth_root,multipoint_evaluation,differential_cryptanalysis
0,0,0,1000,1,2.3805,implementation arithmetic math,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,1,1001,1,1.3998,implementation arithmetic math,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2,2,1271,1,3.0038,arbitrary_precision arithmetic math,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,3,3,1550,4,1.5407,math implementation,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,4,4,2338,1,1.9607,arbitrary_precision arithmetic math,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9717,9717,9717,23132,28,2.5000,generating_function,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9718,9718,9718,23467,28,1.0000,,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9719,9719,9719,23667,28,1.0000,,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9720,9720,9720,23679,28,1.6667,binary_search bipartite_matching game_...,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [103]:
# 사용할 Column
# problemid, level, averageTries, tag-key
# 데이터 전처리
# problem = problem.iloc[:,1:]

In [104]:
# 새로운 피벗 테이블을 사용하기 위해 Column 명을 변경
user.columns = ['userId', 'problemId']

In [105]:
# 데이터 중복행 삭제
problem = problem.drop_duplicates()
# K-Means를 하기 위해 문제 아이디를 제거
problem_no = problem.iloc[:,1:]

In [106]:
# 30개의 단계로 구성되었으므로 30개로 작성
k = 30
model = KMeans(n_clusters=k, random_state=10)
model.fit(problem_no)

ValueError: could not convert string to float: '  implementation    arithmetic    math  '

In [ ]:
# Cluseter의 결과를 저장
problem['cluster'] = model.fit_predict(problem_no)

In [ ]:
problem.tail()

In [ ]:
# 사용자와 문제정보를 병합
user_problem = pd.merge(user, problem, on="problemId")

In [107]:
# 사용자와 문제에 대한 피벗 테이블을 생성
user_problem_pi = user_problem.pivot_table('cluster', index='problemId', columns='userId')

In [108]:
# 백준에는 있지만 Solved ac에는 없는 문제를 -1로 태깅
user_problem_pi.fillna(-1,inplace=True)
user_problem_pi.head()

userId,0000000000,0000064,0000sb,0004poppop,000ian,001,001dltmdals,0021,006ws,007001as,...,zzzwldy,zzzxc123,zzzz01111,zzzz465,zzzzddz,zzzzll,zzzzong123,zzzzz9887,zzzzzzzz,zzzzzzzzu2
problemId,,,,,,,,,,,,,,,,,,,,,
1000,4.0,4.0,4.0,4.0,-1.0,4.0,-1.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,-1.0,4.0,4.0,4.0,4.0,4.0
1001,4.0,4.0,4.0,4.0,-1.0,4.0,-1.0,-1.0,4.0,-1.0,...,4.0,4.0,4.0,4.0,-1.0,4.0,-1.0,4.0,4.0,4.0
1002,28.0,-1.0,-1.0,-1.0,-1.0,28.0,-1.0,28.0,-1.0,-1.0,...,28.0,-1.0,-1.0,28.0,-1.0,28.0,-1.0,-1.0,28.0,-1.0
1003,16.0,-1.0,-1.0,-1.0,-1.0,16.0,-1.0,16.0,16.0,-1.0,...,16.0,-1.0,-1.0,16.0,-1.0,16.0,-1.0,-1.0,-1.0,-1.0
1004,-1.0,-1.0,-1.0,-1.0,-1.0,16.0,-1.0,16.0,16.0,-1.0,...,16.0,-1.0,-1.0,-1.0,-1.0,16.0,-1.0,-1.0,-1.0,-1.0


In [109]:
# 문제간의 유사도를 구해 새로운 matiricx생성
problem_sim = cosine_similarity(user_problem_pi,user_problem_pi).argsort()[:,::-1]

In [110]:
#문제 추천의 함수
def get_recommand_problem(df, problemId, top=30):
    target_problem_index = df[df['problemId'] == problemId].index.values
    
    # 코사인 유사도중 비슷한 코사인 유사도를 가진 정보를 뽑아낸다.
    sim_index = problem_sim[target_problem_index, :top].reshape(-1)
    # 본인 제외
    sim_index = sim_index[sim_index != target_problem_index]
    
    result = df.iloc[sim_index].sort_values('level', ascending=True)[:3]
    return result

In [111]:
get_recommand_problem(problem,14491)

,Unnamed: 0.1,Unnamed: 0,problemId,level,averageTries,tag-key,math,implementation,dp,graphs,...,top_tree,chordal_graph,majority_vote,a_star,rb_tree,birthday,degree_sequence,discrete_kth_root,multipoint_evaluation,differential_cryptanalysis
190,190,190,24356,2,1.6146,arithmetic math,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
469,469,469,23971,3,1.5387,arithmetic math,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
856,856,856,23802,3,1.3411,implementation,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [112]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

from sklearn.metrics.pairwise import cosine_similarity

def init():
    # 문제에 대한 정보
    problem = pd.read_csv('data/tag_key.csv')
    user = pd.read_csv('data/userSolved.csv')
    user.columns = ['userId', 'problemId']

    k = 50
    model = KMeans(n_clusters=k, random_state=10)
    model.fit(problem_no)

    # Cluseter의 결과를 저장
    problem['cluster'] = model.fit_predict(problem_no)

    # 사용자와 문제정보를 병합
    user_problem = pd.merge(user, problem, on="problemId")

    # 사용자와 문제에 대한 피벗 테이블을 생성
    user_problem_pi = user_problem.pivot_table('cluster', index='problemId', columns='userId')

    # 백준에는 있지만 Solved ac에는 없는 문제를 -1로 태깅
    user_problem_pi.fillna(-1,inplace=True)

    # 문제간의 유사도를 구해 새로운 matiricx생성
    problem_sim = cosine_similarity(user_problem_pi,user_problem_pi).argsort()[:,::-1]



def Cal_distance(result, target_index):
        sorter = []
        for i in range(len(result)):
            sorter.append(pow(result.iat[i,1] - self.problem.iat[target_index,1],2))
        result['sorter'] = sorter
        return result
     
def get_recommand_problem(problemId, top=30):
    for i in range(len(problem)):
        if problem['problemId'][i] == problemId:
            target_problem_index = i
            break

    # 코사인 유사도중 비슷한 코사인 유사도를 가진 정보를 뽑아낸다.
    sim_index = self.problem_sim[target_problem_index, :top].reshape(-1)
    # 본인 제외
    sim_index = sim_index[sim_index != target_problem_index]

    result = problem.iloc[sim_index]
    result = Cal_distance(result, target_problem_index)
    return result.sort_values('sorter')[:5]


In [113]:
problem_sim

array([[   0,    1,  421, ...,  890, 1459, 3755],
       [   1,    0, 2179, ...,  890,  889, 3755],
       [   2,  596,  433, ..., 1387, 1459, 3755],
       ...,
       [4867, 4865, 4765, ...,  484,    0,  421],
       [4868, 4869, 4801, ...,  484,    0,  421],
       [4869,  919, 1589, ...,  484,    0,  421]])

In [114]:
type(problem['problemId'][1])

numpy.int64

In [115]:
#하나의 클래스로 진행
class Recommand:
    
    def __init__(self):
        # 사용자와 문제 정보를 읽어 저장
        self.problem = pd.read_csv('data/tag_key.csv')
        self.user = pd.read_csv('data/userSolved.csv')
        # 의미 없는 열 제거
#         self.problem = self.problem.iloc[:,1:]
        # 새로운 피벗 테이블을 사용하기 위해 Column 명을 변경
        self.user.columns = ['userId', 'problemId']
        # 데이터 중복행 삭제
        self.problem = self.problem.drop_duplicates()
    
    def Start_KMeans(self):
        # K-Means를 하기 위해 문제 아이디를 제거
        problem_no = self.problem.iloc[:,1:]
        # Cluster는 Solved.ac의 난이도의 수로 설정
        k = 30
        # 모델을 생성
        model = KMeans(n_clusters=k, random_state=10)
        # K-Means를 실행
        model.fit(problem_no)
        self.problem['cluster'] = model.fit_predict(problem_no)
        # 사용자와 문제정보를 병합
        user_problem = pd.merge(self.user, self.problem, on="problemId")
        # 사용자와 문제에 대한 피벗 테이블을 생성
        user_problem_pi = user_problem.pivot_table('cluster', index='problemId', columns='userId')
        # 백준에는 있지만 Solved ac에는 없는 문제를 -1로 태깅
        user_problem_pi.fillna(-1,inplace=True)
        # 사용자와 문제정보를 병합
        self.problem_sim = cosine_similarity(user_problem_pi, user_problem_pi).argsort()[:,::-1]
        
    def Cal_distance(self, result, target_index):
        sorter = []
        for i in range(len(result)):
            sorter.append(pow(result.iat[i,1] - self.problem.iat[target_index,1],2))
        result['sorter'] = sorter
        return result
     
    def get_recommand_problem(self, problemId, top=30):
        for i in range(len(self.problem)):
            if self.problem['problemId'][i] == problemId:
                target_problem_index = i
                break
        
        # 코사인 유사도중 비슷한 코사인 유사도를 가진 정보를 뽑아낸다.
        sim_index = self.problem_sim[target_problem_index, :top].reshape(-1)
        # 본인 제외
        sim_index = sim_index[sim_index != target_problem_index]
    
        result = self.problem.iloc[sim_index]
        result = self.Cal_distance(result, target_problem_index)
        return result.sort_values('sorter')[:5]


In [116]:
rc = Recommand()
rc.Start_KMeans()

ValueError: could not convert string to float: '  implementation    arithmetic    math  '

In [ ]:
rc.get_recommand_problem(1001)